In [1]:
#! /usr/bin/python
# -*- coding: utf-8 -*-
# @author izhangxm
# Copyright 2017 izhangxm@gmail.com. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [21]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from itertools import product
import os.path as osp
from scipy.optimize import leastsq
import time
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import math
import sunode
import sunode.wrappers.as_pytensor
from copy import deepcopy

# from tqdm import tqdm, trange
from tqdm.notebook import  tqdm
from IPython.display import display as print
from datetime import datetime

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

from core_lib import MyDataset, get_format_time, plot_dataset, get_dcdt_func_for_sunode, get_dcdts_for_scipy_odeint, get_dcdts_for_solve_ivp
from core_lib import get_predict_ks
from core_lib import r2_loss



In [103]:
# simulator function
def competition_model(rng, t_eval, y0,  ks, k_kinetics, size=None):
    # print(y0)
    y = odeint(get_dcdts_for_scipy_odeint(), y0=y0, t=t_eval, args=(ks, k_kinetics))
    return y

In [92]:

def get_model(dataset, k_kinetics, k_sigma_priors=0.01, kf_type=0):

    df = dataset.get_df()
    times = df['time'].values
    
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    cct_names, error_names = dataset.get_var_col_names()
        
    # 定义参数优化模型
    mcmc_model = pm.Model()
    ## 参数个数
    params_n = 11
    parames ={}
    
    with mcmc_model:
        for ki in range(1, params_n + 1):
            if kf_type == 0:
                p_dense = pm.HalfNormal(f"k{ki}", sigma=k_sigma_priors)
            else:
                p_dense = pm.Normal(f"k{ki}",mu=0, sigma=k_sigma_priors)
            parames[f"k{ki}"] = (p_dense, ())
    
    # parames['k_kinetics'] = np.array(k_kinetics, dtype=np.float64)
    parames['extra']=  np.zeros(1)
    
    c0 = {}
    with mcmc_model:
        for c_name in cct_names:
            _maxx = df[c_name].values.max()
            c0[f"{c_name}"] = (pm.HalfNormal(f"{c_name}_s", sigma=_maxx), ())
            # c0[f"{c_name}"] = (pm.Normal(f"{c_name}_s", mu=_meanx, sigma=s), ())
    
        y_hat, _, problem, solver, _, _ = sunode.wrappers.as_pytensor.solve_ivp(
            y0=c0,
            params=parames,
            rhs=get_dcdt_func_for_sunode(k_kinetics),
            tvals=times,
            t0=times[0],
        )
        
        sd = pm.HalfNormal('sd')
        for c_name in cct_names:
            pm.Normal(f'{c_name}', mu=y_hat[f"{c_name}"], sigma=sd, observed=df[f"{c_name}"].values)
            pm.Deterministic(f'{c_name}_mu', y_hat[f"{c_name}"])
    return mcmc_model

In [94]:
# k_kinetics = np.repeat(1, 11).astype(np.uint8)
k_kinetics = np.array([0,1,1,1,1,0,0,1,1,1,0])
# ks = np.random.random(11)/100 # 先验k
ks = np.array([0.00071942, 0.00269696, 0.00498945, 0.00444931, 0.00571299, 0.00801272, 0.00131931, 0.00319959, 0.00415571, 0.00228432, 0.00177611])
#  =======================================================

# t_eval = np.arange(0, 150, 5)

dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, error_names = dataset.get_var_col_names()

# c0 = df[cct_names].iloc[0].values
# dataset.set_as_sim_dataset(dcdt_func_for_odeint, t_eval, c0, args=(ks, k_kinetics))
# df = dataset.get_df()

# plot_dataset(dataset=dataset)
mcmc_model = get_model(dataset, k_kinetics, k_sigma_priors=0.01, kf_type=0)


In [ ]:
plot_dataset(dataset=dataset)

In [ ]:
idata = pm.sample(draws=1000, model=mcmc_model, chains=4, cores=4)

In [104]:
def get_result(dataset, idata):
    res_df = az.summary(idata)
    
    predict_ks = get_predict_ks(idata=idata)
    df = dataset.get_df()
    times = df['time'].values
    
    cct_names, error_names = dataset.get_var_col_names()
    
    c_df = pd.DataFrame(columns=cct_names)
    t_n = len(times)
    for c_name in cct_names:
        c_value = []
        for i in range(t_n):
            _x = res_df['mean'][f"{c_name}_mu[{i}]"]
            c_value.append(_x)
        c_df[c_name] = c_value
    
    return predict_ks, c_df

predict_ks, c_df = get_result(dataset, idata)

print(predict_ks)
print(c_df[cct_names])
# print(az.summary(idata))

array([9.46627500e-04, 1.80282300e-04, 7.75364680e-03, 2.25054380e-03,
       9.93785290e-03, 1.11419580e-03, 2.06023280e-03, 1.56184350e-03,
       1.66389505e-02, 6.79560000e-05, 1.19875624e-02])

,xNH3,xNO3,xNO2,xNOrg,xN2,ANH3,ANO3,ANO2,ANOrg,AN2
0,0.924,2.851,0.069,62.093,0.000,8.082,0.621,0.060,0.059,0.026
1,0.649,2.862,0.155,62.135,0.068,4.992,0.548,0.320,0.124,0.651
2,0.538,2.903,0.173,62.011,0.156,2.712,0.498,0.321,0.151,0.534
3,0.492,2.950,0.178,61.824,0.246,1.413,0.460,0.330,0.162,0.456


In [ ]:
c_df['time']  = df['time'].values
dataset_new = MyDataset("dataset/data.csv")
dataset_new.set_dataset(c_df)
plot_dataset(dataset, dataset_pred=dataset_new)


In [106]:
r2_all = r2_loss(dataset.get_cct(), c_df[cct_names].values)

for c_name in cct_names:
    _loss = r2_loss(dataset.df[c_name].values, c_df[c_name].values)
    print(f"r2: {c_name} {_loss}")


0.9992822993532159

'r2: xNH3 -0.8926770700262396'

'r2: xNO3 -85.8514520337752'

'r2: xNO2 -5.94321655742469'

'r2: xNOrg -13.762082355263884'

...

'r2: ANH3 0.9865236241877875'

'r2: ANO3 -448.7425181981374'

'r2: ANO2 -2.4792918569177638'

'r2: ANOrg -3.5620796821307694'

'r2: AN2 -2.8811935399551487'

In [ ]:
k_kinetics = np.array([0,1,1,1,1,0,0,1,1,1,0])

ks = predict_ks
t_eval = np.arange(0, 150, 1)
c0 = df[cct_names].iloc[0].values
cs = odeint(get_dcdts_for_scipy_odeint(), c0, t=t_eval, args=(ks, k_kinetics))
# res = odeint(func=dcdt_func2, y0=c0, t=t_eval, args=(ks,k_kinetics))
# cs = np.transpose(res, [1,0])

print(cs.shape)

cols = 5
rows = math.ceil(len(cct_names) / cols)

fig, fig_axes = plt.subplots(ncols=cols, nrows=rows, figsize=(4.2 * cols, 4 * rows), dpi=100)
if isinstance(fig_axes, np.ndarray):
    fig_axes = fig_axes.reshape(-1)
else:
    fig_axes = [fig_axes]

for i, axes in enumerate(fig_axes):
    if i >= len(cct_names):
        axes.axis('off')
        continue

    y_name = cct_names[i]
    Y = df[y_name].values
    axes.plot(df['time'].values, Y, '*', label=f"ob")
    axes.set_ylabel(f'cct_{y_name}')
    axes.set_xlabel(f'time(h)')

    # axes.plot(df['time'].values, df[rates_names[i]].values, '+', label=f"rate")
    
    axes.plot(t_eval, cs[:, i], 'r', label=f"c(t)")
    # axes.plot(t_eval, dcdt_df[y_name].values,'g', label=f"c'(t)")

    axes.legend()
    # axes.set_title(f"{y_name}", fontsize=14)

plt.tight_layout()
plt.show()

